# 03-03 Pandas-Musterlösung

## Hinweise zur Übung

Diese Übung orientiert sich an den Aufgaben der vorherigen SQL-Übung, aber Sie setzen alles mit Pandas-Funktionen statt SQL-Abfragen um.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql pandas

In [ ]:
import os
import sys
import urllib.request
import gzip
import shutil
import pandas
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
# Konfiguration
base_url_quellen   = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/08-Datenanalyse+Visualisierung/data"
base_url_reporting = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/dwh/reporting.sqlite.gz", base_url_reporting + "reporting.sqlite.gz")

# Die Sqlite-Datenbank ist aufgrund ihrer Größe gezipped und muss vor der Nutzung noch entpackt werden
with gzip.open(base_url_reporting + "reporting.sqlite.gz", "rb") as f_in:
    with open(base_url_reporting + "reporting.sqlite", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_reporting      = base_url_reporting + "reporting.sqlite"

db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_reporting

## Laden von Daten aus der SQLite-Datenbank in Dataframes

Der Abschnitt zum Laden der Tabellen D_PATIENT, F_FAELLE, D_FALLART und D_DIAGNOSE wird hier 1:1 übernommen.


In [ ]:
# Abfrage der Tabelle D_PATIENT und Ablage in Dataframe df_patient
sql = """
SELECT *
  FROM d_patient
"""
resultset = %sql $db_url_reporting $sql
df_patient = resultset.DataFrame()
df_patient.head()

In [ ]:
# Abfrage der Tabelle F_FAELLE und Ablage in Dataframe df_faelle
sql = """
SELECT *
  FROM f_faelle
"""
resultset = %sql $db_url_reporting $sql
df_faelle = resultset.DataFrame()
df_faelle.head()

In [ ]:
# Abfrage der Tabelle D_FALLART und Ablage in Dataframe df_fallart
sql = """
SELECT *
  FROM d_fallart
"""
resultset = %sql $db_url_reporting $sql
df_fallart = resultset.DataFrame()
df_fallart.head()

In [ ]:
# Abfrage der Tabelle D_DIAGNOSE und Ablage in Dataframe df_diagnose
sql = """
SELECT *
  FROM d_diagnose
"""
resultset = %sql $db_url_reporting $sql
df_diagnose = resultset.DataFrame()
df_diagnose.head()

### Aufgabe: Laden Sie die Tabelle D_ORGA in den Dataframe df_orga

In [ ]:
# Abfrage der Tabelle D_ORGA und Ablage in Dataframe df_orga
sql = """
SELECT *
  FROM d_orga
"""
resultset = %sql $db_url_reporting $sql
df_orga = resultset.DataFrame()
df_orga.head()

## Einfache Abfragen auf Datafames


### Aufgabe: Fragen Sie nur die Spalten `fall_id`, `aufnahme_datum` und `entlass_datum` des Dataframes `df_faelle` ab

In [ ]:
# Nur die Spalten fall_id, aufnahme_datum & entlass_datum ausgeben
df_faelle[["fall_id", "aufnahme_datum", "entlass_datum"]].head()

### Aufgabe: Fragen Sie männliche Patienten aus dem Datafram `df_patient` ab (alle Spalten)

In [ ]:
# Dataframe df_patient auf männliche filtern und erste Zeilen ausgeben
df_patient.query("patient_geschlecht == 'M'").head()

## Abfragen über mehrere verbundene Dataframes

### Aufgabe: Mergen Sie die Dataframes `df_patient` und `df_faelle` und zeigen Sie die ersten Zeilen (alle Spalten) an

In [ ]:
# Dataframes df_patient und df_faelle über Schlüsselspalte fall_id mergen und erste Zeilen ausgeben
df_patient.merge(df_faelle, on="patient_id").head()

### Aufgabe: Ergänzen Sie die Abfrage um einen INNER JOIN mit dem Dataframe `df_fallart'

Geben Sie nur die folgenden Spalten aus:
* df_patient: patient_id, patient_nachname
* df_faelle: aufnahme_datum, entlass_datum
* df_fallart: fallart_name

💡 Hinweis: es kann sein, dass Sie Spalten, die als Join-Kriterien benötigt werden, zunächst einbeziehen und später per `drop()`-Methode wieder entfernen müssen, da anders als bei SQL-Datenbanken nur Spalten genutzt werden können, die in dieser Phase der Pipeline auch noch im Dataframe vorhanden sind. In SQL kann dagegen für die Join-Kriterien auf alle Spalten zugegriffen werden, auch wenn sie im `SELECT`-Abschnitt nicht aufgeführt sind.

In [ ]:
# Vorherige Abfrage um merge zu Dataframe df_fallart (über fallart_id) erweitern
# und nur die vorgegebenen Spalten auslesen
df_patient[["patient_id", "patient_nachname"]] \
  .merge(df_faelle[["patient_id", "fallart_id", "aufnahme_datum", "entlass_datum"]], on="patient_id") \
  .merge(df_fallart, on="fallart_id") \
  .drop(columns="fallart_id") \
  .head()

### Aufgabe: Ergänzen Sie die vorherige Abfrage um einen LEFT OUTER JOIN, um den Bezeichner der Hauptdiagnose anzuzeigen

Achten Sie darauf, dass Fälle mit fehlender Hauptdiagnose weiterhin sichtbar bleiben. Geben Sie dazu das gesamte Ergebnis aus, statt mit der `head()`-Methode nur auf die ersten Zeilen zu beschränken. Sie können das Tabellen-Icon rechts neben dem Output nutzen, um eine interaktiv blätter- und sortierbare Anzeige zu erhalten.

In [ ]:
# Vorherige Abfrage um merge zu Dataframe d_diagnose (über hauptdiagnose_snomed_id/snomed_id) erweitern
# und nur die vorgegebenen Spalten auslesen
df_patient[["patient_id", "patient_nachname"]] \
  .merge(df_faelle[["patient_id", "fallart_id", "aufnahme_datum", "entlass_datum", "hauptdiagnose_snomed_id"]], on="patient_id") \
  .merge(df_fallart, on="fallart_id") \
  .merge(df_diagnose[["snomed_id", "snomed_name"]], left_on="hauptdiagnose_snomed_id", right_on="snomed_id", how="left") \
  .drop(columns=["fallart_id", "hauptdiagnose_snomed_id"])

### Aufgabe: Geben Sie Einrichtungen aus, die keine Behandlungsfälle hatten

Die Entsprechung des `IS NULL`-Operators, der in SQL zur Selektion leerer Spalten verwendet wird, ist die `isna()`-funktion, die in der `query()`-Methode verwendet werden kann.

In [ ]:
# LEFT OUTER Merge des Dataframe df_orga mit dem Dataframe df_faelle
# und Filterung auf leere Fall-Datensätze
df_orga.merge(df_faelle, on="einrichtung_id", how="left")\
  .query("fall_id.isna()")

## Aggregation von Daten

In den folgenden Abfragen benötigen Sie Gruppierungsmerkmale und Aggregatfunktionen, um Daten über mehrere Tabellenzeilen hinweg zusammengefasst auszuwerten.

### Aufgabe: Berechnen Sie den aufsummierten Gesamterlös pro Einrichtung

Geben Sie dazu die die folgenden Spalten aus:
* df_orga: `standort_name`,  `einrichtung_name`
* df_faelle: Summe von `erloes_fallpauschale`

In [ ]:
# Dataframes df_orga und df_faelle (über "einrichtung_id") mergen,
# anschließend die nur für den Merge nötige Spalte "einrichtung_id" droppen
# danach über die Spalten "standort_name" und "einrichtung_name" gruppieren
# und "erloes_fallpauschale" summieren
df_orga[["standort_name", "einrichtung_id", "einrichtung_name"]] \
  .merge(df_faelle[["einrichtung_id", "erloes_fallpauschale"]]) \
  .drop(columns="einrichtung_id") \
  .groupby(["standort_name", "einrichtung_name"]) \
  .agg({'erloes_fallpauschale': 'sum'})

### Aufgabe: Ergänzen Sie die Abfrage um die Fallzahl und den mittleren Erlös und filtern auf stationäre Fälle

Hinweise:
* stationäre Fälle haben die Fallart "inpatient"
* speichern Sie das Ergebnis in einen neuen Dataframe "df_auswertung", damit wir es in den folgenden Aufgaben weiternutzen können.

In [ ]:
# Vorherige Abfrage um merge mit Dataframe df_fallart ergänzen,
# auf Fallart "inpatient" filtern und das Ergebnis in den Dataframe "df_auswertung" speichern
df_auswertung = df_orga[["standort_name", "einrichtung_id", "einrichtung_name"]] \
  .merge(df_faelle[["einrichtung_id", "fallart_id", "erloes_fallpauschale"]]) \
  .merge(df_fallart, on="fallart_id") \
  .query("fallart_name == 'inpatient'") \
  .drop(columns=["einrichtung_id", "fallart_id", "fallart_name"]) \
  .groupby(["standort_name", "einrichtung_name"]) \
  .agg(fallzahl=("einrichtung_name", "size"), erloes_sum=("erloes_fallpauschale", "sum"), erloes_avg=("erloes_fallpauschale", "mean"))
df_auswertung

### Aufgabe: Ergänzen Sie die vorherige Abfrage um einen Filter auf Einrichtungen mit mehr als 50 Fällen und sortieren das Ergebnis absteigend nach Gesamterlös

In [ ]:
# Ergebnis der vorherigen Abfrage auf Einträge mit mehr als 50 Fällen filtern
# und absteigend nach Erlös sortieren
df_auswertung.query("fallzahl > 50") \
  .sort_values(by="erloes_sum", ascending=False)

## Pivotieren von Daten zwischen "wide"- und "long"-Formaten

Wir nutzen die `melt()`- und `pivot()`-Funktionen von Pandas, um den Dataframe `df_auswertung`, der ein "wide"-Format hat, in ein "long"-Format zu transformieren und anschließend wieder in ein "wide"-Format umzuklappen.

### Auswertung in "long"-Format transformieren

Hinweise:
* Anders als in der Demo enthält der Dataframe zwei identifizierende Spalten (standort_name, einrichtung_name).
* Die Bezeichner der Kennzahlen sollen in der Spalte "kennzahl" und die Inhalte in der Spalte "wert"! abgelegt werden.
* 🛑 Aufgrund der vorher durchgeführten Aggregation liegt der Dataframe als "Series"-Objekt vor. Dieses muss zunächst mit der Methode `reset_index()` in einen regulären Dataframe umgewandelt werden, damit die `melt()`-methode angewendet werden kann.
* Speichern Sie das Ergebnis im neuen Dataframe "df_auswertung_long"

In [ ]:
# Spalten standort_name und einrichtung_name als identifizierende beibehalten,
# restliche Spalten umklappen und dazu die Spaltennamen in die Spalte "kennzahl" übertragen
# und die Ausprägungen in die Spalte "wert"; Ergebnis in Dataframe df_auswertung_long ablegen
df_auswertung_long = df_auswertung.reset_index() \
  .melt(id_vars=["standort_name", "einrichtung_name"], var_name="kennzahl", value_name="wert")
df_auswertung_long

### Auswertung vom "long" wieder ins "wide"-Format umklappen

In [ ]:
# Spalten standort_name & einrichtung_name als Index übernehmen,
# Bezeichner aus Spalte "kennzahl" und Werte aus Spalte "wert" übernehmen
df_auswertung_long.pivot(index=["standort_name", "einrichtung_name"], columns="kennzahl", values="wert")